In [1]:
# 산출속도 향상을 위해 dataframe을 dict or list 구조로 변경하여 처리함

import sys
import pandas as pd
import numpy as np
import time, datetime
import json


t0 = time.time()

#입력포맷
my_input = {'start_date' : '2018-01-01',
            'init_index' : 1000,
            'init_amt' : 1000000,
            'portfolio' : ['069500','261220','332940','371450','HERO','SPY'],
            'rebalancing' : {
                '2018-01-01' : [15,15,20,25,15,10],
                '2019-01-01' : [15,15,20,25,15,10],
                '2020-01-01' : [15,15,20,25,15,10],
                '2021-01-01' : [15,15,20,25,15,10]}                
            }

prc_list = []
for item in my_input['portfolio'] :
    # excel 파일 읽기
    infile = 'price_' + item + '.xlsx'
    df = pd.read_excel(infile,dtype='unicode')
    
    # timeseries로 변환
    df.set_index('DATE', inplace=True)
    df.index = pd.to_datetime(df.index)
    
    # 컬럼명 'PRICE'를 symbol code로 바꾼다
    df.rename(columns={'PRICE':item},inplace=True)
    df[item] = df[item].astype(float)
    prc_list.append(df)

t1 = time.time()

# 벤치마크 설정
infile = "benchmark.xlsx"
tmp_bm_df = pd.read_excel(infile,dtype='unicode')
    
# timeseries로 변환
tmp_bm_df.set_index('DATE', inplace=True)
tmp_bm_df.index = pd.to_datetime(tmp_bm_df.index)
# 컬럼명 'benchmark'으로 변환
tmp_bm_df.rename(columns={'PRICE':'benchmark'},inplace=True)
tmp_bm_df['benchmark'] = tmp_bm_df['benchmark'].astype(float)

# 벤치마크를 prc_list에 넣는 이유는, 날짜별로 발생할 수 있는 결측치를 없애기 위해서임.
# 예를들어 한국종목으로 포트폴리오를 구성하고, BM을 S&P 500으로 설정한다면, 분석시 데이터 처리가 번거로워진다
prc_list.append(tmp_bm_df)


# 무위험이자율 설정
infile = "riskfree.xlsx"
tmp_rskfre_df = pd.read_excel(infile,dtype='unicode')
# timeseries로 변환
tmp_rskfre_df.set_index('DATE', inplace=True)
tmp_rskfre_df.index = pd.to_datetime(tmp_rskfre_df.index)
# 컬럼명 'BM'으로 변환
tmp_rskfre_df.rename(columns={'CD91':'riskfree'},inplace=True)
tmp_rskfre_df['riskfree'] = tmp_rskfre_df['riskfree'].astype(float)

prc_list.append(tmp_rskfre_df)

# 마켓지표 설정
infile = "kospi.xlsx"
tmp_kospi_df = pd.read_excel(infile,dtype='unicode')
# timeseries로 변환
tmp_kospi_df.set_index('DATE', inplace=True)
tmp_kospi_df.index = pd.to_datetime(tmp_kospi_df.index)
# 컬럼명 'BM'으로 변환
tmp_kospi_df.rename(columns={'PRICE':'kospi'},inplace=True)
tmp_kospi_df['kospi'] = tmp_kospi_df['kospi'].astype(float)

prc_list.append(tmp_kospi_df)



# 하나의 DATAFRAME으로 합친다
prc_df = pd.concat(prc_list,join='outer',axis=1)

# 결측값(N/A)을 처리한다. method='pad'를 넣으면 N/A부분을 직전값으로 대체한다. 
# 상장이 늦게된종목의 경우 앞에서 부터 계속 N/A이므로 결측값의 대체가 어렵다 (나중에 0으로 바꿀지 고민해보자)
prc_df.fillna(method='pad',inplace=True)

# prc_df에서 [벤치마크/무위험이자율/KOSPI]와 포트폴리오 시계열을 분리한다
#bm_series = prc_df['BM']  # Series 로 변환됨
#bm_df = bm_series.to_frame(name='BM') # Datafram으로 바꾸고...
new_df = prc_df[['benchmark','riskfree','kospi']]

#만약 벤치마크 앞쪽데이터가 없을 경우엔 데이터가 있는 날짜의 데이터로 채운다.
#시계열 분석시 직전 데이터로 채워야하나(method='pad'), 힘빼지말자...
new_df.fillna(method='bfill',inplace=True)
#벤치마크를 지수와 같은 스케일(첫시작을 1000)로 변환한다.
ratio = my_input['init_index'] / new_df.iloc[0].loc['benchmark']
new_df['BM1000'] = new_df['benchmark'] * ratio


# 포트폴리오 시계열에서 BM 삭제
prc_df.drop(['benchmark','riskfree','kospi'],axis=1,inplace=True)

# EXCEL에 쓰기 
# df.index = df.index.date은 concat 후에 변환하자. concat 전에 하면, 늦게 상장한 종목부터 합쳐진다
# prc_df.index = prc_df.index.date

# 수익률 구하기
rtn_df = prc_df.pct_change()
rtn_df.fillna(0,inplace=True) # 첫날의 수익률은 0으로 SET


# 초기비중설정 (리벨런싱 반영)
init_wght_df = pd.DataFrame(data=None, columns=rtn_df.columns, index=rtn_df.index)
for key, value in my_input['rebalancing'].items() :
    myindex = init_wght_df.index[init_wght_df.index.get_loc(key,method='bfill')]
    init_wght_df.loc[myindex] = [ x / 100 for x in value ]
    
t2 = time.time()

# 지수 백테스팅

## 초기값 세팅
my_index = my_input['init_index']
my_deposit = my_input['init_amt']

tot_rtn_list = []
my_index_list = []
my_deposit_list = []

##빠른 loop처리를 위해 dict로 변환 후 iteration 작업 (df -> dict -> list)
rtn_dict = rtn_df.to_dict('split')
rtns = rtn_dict['data']

wght_dict = init_wght_df.to_dict('split')
wghts = wght_dict['data']

## 지수 계산 시작
for i in range(0,len(rtns)) :
    tot_rtn = sum([ x*y for (x,y) in zip(rtns[i], wghts[i]) ]) / sum(wghts[i])
    my_index = my_index * (1+tot_rtn)
    my_deposit = my_deposit * (1+tot_rtn)
    
    tot_rtn_list.append(tot_rtn)
    my_index_list.append(my_index)
    my_deposit_list.append(my_deposit)
    
    # 다음 wghts 값이 미리설정되지 않았다면, 수익률을 반영한 비중을 계산한다. 즉, 리벨런싱은 skip
    if i < ( len(rtns) -1 ) :
        if  np.isnan(wghts[i+1]).any() :
            wghts[i+1] = [ (1+x)*y for (x,y) in zip(rtns[i],wghts[i])]
        #else :
        #    print (wghts[i+1])

index_df = pd.DataFrame({'backtest':my_index_list, 'rtn':tot_rtn_list, 'deposit':my_deposit_list},index=rtn_df.index)
weight_df =  pd.DataFrame(data=wghts, columns=init_wght_df.columns, index=init_wght_df.index)
t3 = time.time()

# 시계열분석을 위해, 산출된 지수값과 벤치마크, 무위험이자율, kospi지수를 json형태로 변환한다.

result_df = pd.concat([index_df,new_df],join='outer',axis=1)
result_df.reset_index(inplace=True)
result_df['DATE'] = result_df['DATE'].dt.strftime('%Y-%m-%d')
result_df.to_json('result.json',orient='records')
t4 = time.time()

print("백테스트 완료!!")

elapsed_time = t1 - t0
print(f"종목별가격 로드 : {elapsed_time} sec")

elapsed_time = t2 - t1
print(f"수익률계산 및 비중 초기화 : {elapsed_time} sec")

elapsed_time = t3 - t2
print(f"지수 백테스트 : {elapsed_time} sec")

elapsed_time = t4 - t3
print(f"결과데이터변환 : {elapsed_time} sec")


elapsed_time = t4 - t0
print(f"총 소요시간 : {elapsed_time} sec")


index_df.to_excel("myindex_2.xlsx")

/Users/sunkism/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
ipykernel_launcher:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
백테스트 완료!!
종목별가격 로드 : 0.14502215385437012 sec
수익률계산 및 비중 초기화 : 0.1700139045715332 sec
지수 백테스트 : 0.025710105895996094 sec
결과데이터변환 : 0.010991096496582031 sec
총 소요시간 : 0.35173726081848145 sec


In [48]:
import sys
import json
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy import stats
from scipy.stats.mstats import gmean
import simplejson

def init_analyze(input_df) :
    df = input_df.copy()
    df.rename(columns={'DATE':'date'},inplace=True)
    df.set_index(['date'],inplace=True)
    df.index = pd.to_datetime(df.index)
    df['prev'] = df['backtest'].shift(1)
    df['kospi_rtn'] = df['kospi'].pct_change()
    df['bm_prev'] = df['benchmark'].shift(1)
    df['bm_rtn'] = df['benchmark'].pct_change()
    df['riskfree'] = df['riskfree'] / 100

    # 시계열 분석 편의를 위해 첫데이터를 날린다.
    df = df.iloc[1:]
    return df

def init_result():
    return {
    'final_balance' : 0,
    'cagr' : 0,
    'stdev' : 0,
    'annlzd_stdev' : 0,
    'arith_mean' : 0,
    'annlzd_arith_mean' : 0,
    'geo_mean' : 0,
    'annlzd_geo_mean' : 0,
    'vol' : 0,
    'annlzd_vol' : 0,
    'hist_var' : 0,
    'anal_var' : 0,
    'c_var' : 0,
    'best_y' : {'year' : 0, 'rtn' : 0},
    'worst_y' : {'year' : 0, 'rtn' : 0},
    'mdd' : 0,
    'skewness' : 0,
    'kurtosis' : 0,
    'sharpe_rto' : 0,
    'sortino_rto' : 0,
    'down_dev' : 0,
    'vs_market' : {'beta' : 0, 'alpha' : 0, 'r2' : 0, 'corr' : 0},
    'vs_benchmark' : {'beta' : 0, 'alpha' : 0, 'r2' : 0, 'corr' : 0}
    }

def analyze_data(df):
    
    result = init_result()
    
    NUM_OF_DAYS = 252
    
    # 1. Final Balance
    #initial_balance = 1000000
    #final_balance = initial_balance * df['backtest'].iloc[-1] / df['prev'].iloc[0]
    result['final_balance'] = df['deposit'].iloc[-1]

    # 2. CAGR (2000-12월 종가부터, 2019-04월 종가까지 기간을 잡는다.)
    year = df['backtest'].count() / NUM_OF_DAYS
    # 2000-12월 종가는 2001-1월의 전일가격을 사용
    CAGR = ( df.iloc[-1]['backtest'] / df.iloc[0]['prev'] )**(1/year) - 1
    result['cagr'] = CAGR

    # 3. Stdev (Annualized standard deviation of monthly returns)
    stdev = df['rtn'].std()
    result['stdev'] = stdev
    # 연간화를 위해 루트 NUM_OF_DAYS 를 곱한다.
    annlzd_stdev = stdev*(NUM_OF_DAYS**0.5)
    result['annlzd_stdev'] = annlzd_stdev
    result['vol'] = stdev
    result['annlzd_vol'] = annlzd_stdev

    # 4. Arithmetic Mean (monthly). 
    arith_mean = df['rtn'].mean()
    result['arith_mean'] = arith_mean

    # 5. Arithmetic Mean (annualized).
    annualized_arith_mean = (1 + arith_mean) ** NUM_OF_DAYS - 1
    result['annlzd_arith_mean'] = annualized_arith_mean

    # 6. Geometric Mean, scipy 의 gmean사용
    # 수익률의 기하평균은 각 수익률에 1을 더한후 루트를 적용, 이후에 1을 뺀다
    # monthly_rtn의 모든 컬럼값에 1을 더한다
    df['rtn_1'] = df['rtn'] + 1
    # gmean은 list형의 인자를 받는다
    geo_mean = gmean(df['rtn_1'].tolist()) - 1
    result['geo_mean'] = geo_mean

    # 7. Geometric Mean(annualized)
    annualized_geo_mean = ( 1 + geo_mean) ** NUM_OF_DAYS - 1
    result['annlzd_geo_mean'] = annualized_geo_mean

    # 8. Volatility (monthly) . 변동성은 표준편차를 의미
    #stdev = m_idx['rtn'].std() 
    #result['stdev'] = stdev

    # 9. Volatility (annualized). 3에서 구한 Stdev와 같은 값이다
    # 연간화를 위해 루트12 를 곱한다.
    #stdev = stdev*(12**0.5)

    # 10. VaR
    # 10.1 Historical VaR 
    # exclusive quantile을 자체 구현
    def quantile_exc(df2, q):
        list_sorted = sorted(df2) # sorted()는 list형의 결과를 리턴한다
        rank = q * (len(list_sorted) + 1) - 1
        #print ("q_exc : ", rank)
        #assert rank > 0, 'quantile is too small'
        if rank < 0 :
            print ('quantile is too small')
            return 0
        rank_l = int(rank)
        return list_sorted[rank_l] + (list_sorted[rank_l + 1] - 
                                      list_sorted[rank_l]) * (rank - rank_l)

    historical_var_95 = quantile_exc(df['rtn'], 0.05)
    if (historical_var_95 == 0) :
        historical_var_95 = df['rtn'].quantile(0.05)
    result['hist_var'] = historical_var_95

    # 10.2 Analytical VaR
    mean = df['rtn'].mean()
    stdev = df['rtn'].std()
    analytical_var_95 = norm.ppf(0.05, mean, stdev)
    result['anal_var'] = analytical_var_95

    # 10.3 Conditional VaR
    # 자체구현
    def conditional_var(df3, q):
        list_sorted = sorted(df3)
        rank = q * len(list_sorted) 
        rank_l = int(rank)

        sum_rtn = 0
        sum_rtn = sum(i for i in list_sorted[0:rank_l])

        return 1 / rank * sum_rtn

    cvar_95 = conditional_var(df['rtn'], 0.05)
    result['c_var'] = cvar_95

    # 11. Best Year / Worst Year
    # 년단위 데이터로 resamplingn
    y_idx = df.resample(rule='Y').last()
    y_idx['rtn'] = y_idx['backtest'].pct_change()
    if len(y_idx) == 1 :
        min_val = df['backtest'].iloc[-1] / df['prev'].iloc[0] - 1
        min_idx = df['backtest'].idxmin()
        max_val = df['backtest'].iloc[-1] / df['prev'].iloc[0] - 1
        max_idx = df['backtest'].idxmax()     
    else :
        min_val = y_idx['rtn'].min()
        min_idx = y_idx['rtn'].idxmin()
        max_val = y_idx['rtn'].max()
        max_idx = y_idx['rtn'].idxmax()
    result['best_y']['year'] = max_idx.year
    result['best_y']['rtn'] = max_val
    result['worst_y']['year'] = min_idx.year
    result['worst_y']['rtn'] = min_val

    # 12. MDD, 
    # - step1.지수의 수익률을 일별 누적(1+r을 계속곱해나감). 
    # - step2. 누적수익률에 대한 MAX를 일별로 기록
    # - step3. 일별로 누적수익률과 MAX수익률 간의 차이((CUM - MAX) / MAX) 가 가장 큰 것을 잡는다.

    #  등락률에 1을 더한다
    df['rtn_1'] = df['rtn'] + 1

    # 누적수익률계산
    df['cum'] = df['rtn_1'].cumprod()

    # 누적수익률중 최고값
    df['high'] = df['cum'].cummax()

    # drawdown 계산
    df['drawdown'] = (df['cum'] - df['high'])/df['high']
    MDD = df['drawdown'].min()
    result['mdd'] = MDD

    # 13. Skewness
    skewness = df['rtn'].skew()
    result['skewness'] = skewness

    # 14. Excess Kurtosis
    ex_kurtosis = df['rtn'].kurtosis()
    result['kurtosis'] = ex_kurtosis

    # 15. Ratio
    # https://www.quantnews.com/performance-metrics-sharpe-ratio-sortino-ratio/
    # 15.1 Sharpe Ratio
    # denominator - month(12), day(252)
    denominator = NUM_OF_DAYS
    df['excess_rtn'] = df['rtn'] - df['riskfree']/denominator
    sharpe_rto = df['excess_rtn'].mean() /  df['excess_rtn'].std() * np.sqrt(denominator)
    result['sharpe_rto'] = sharpe_rto

    # 15.2 Sortino Ratio
    target = 0
    df['downside_rtn'] = 0
    df.loc[df['rtn'] < target, 'downside_rtn'] = df['rtn']**2
    down_stdev = np.sqrt(df['downside_rtn'].mean())
    sortino_ratio = df['excess_rtn'].mean()/down_stdev * np.sqrt(denominator)
    result['sortino_rto'] = sortino_ratio
    result['down_dev'] = down_stdev

    # downside_stdev 를 excess_rtn으로 계산
    #m_idx['downside_rtn2'] = 0
    #m_idx.loc[m_idx['excess_rtn'] < target, 'downside_rtn2'] = m_idx['excess_rtn']**2
    #down_stdev2 = np.sqrt(m_idx['downside_rtn2'].mean())
    #sortino_ratio = m_idx['excess_rtn'].mean()/down_stdev2 * np.sqrt(denominator)

    # 16. [vsMarket] Beta, Alpha, R-squared, correlation
    # Beta, Alpha, R squared 참고사이트
    # http://gouthamanbalaraman.com/blog/calculating-stock-beta.html
    # https://stackoverflow.com/questions/893657/how-do-i-calculate-r-squared-using-python-and-numpy

    # 16.1 Beta
    covariance = np.cov(df['rtn'], df['kospi_rtn'])
    # variance는 np.var로 구할수도 있으나, covariance[1,1] 과 같다
    #variance = np.var(m_idx['mkt_rtn'],ddof=1)
    beta = covariance[0,1] / covariance[1,1]
    result['vs_market']['beta'] = beta

    # 16.2 Alpha
    alpha = df['rtn'].mean() - beta*(df['kospi_rtn'].mean())
    #연환산
    y_alpha = (1 + alpha) ** NUM_OF_DAYS - 1
    result['vs_market']['alpha'] = y_alpha

    # 16.3 R squared 
    # R2 - numpy_manual

    ypred = alpha + beta * df['kospi_rtn']
    SS_res = np.sum(np.power(ypred - df['rtn'],2))
    SS_tot = covariance[0,0] * (len(df) - 1) # SS_TOT is sample_variance*(n-1)
    r_squared = 1. - SS_res/SS_tot
    result['vs_market']['r2'] = r_squared

    # 1year momentum (bonus) 
    momentum = np.prod(1+df['rtn'].tail(NUM_OF_DAYS).values) - 1

    # 16.4 correlation
    # 비교를 위해 'rtn', 'mkt_rtn'만 새로운 dataframe 으로 copy
    #new_df = m_idx[['rtn','mkt_rtn']].copy()
    #corr = new_df.corr()
    corr = df['rtn'].corr(df['kospi_rtn'])
    result['vs_market']['corr'] = corr
    
    if 'benchmark' in df.columns:
        
        # 17. [vsBenchmark] Beta, Alpha, R-squared, correlation
        # Beta, Alpha, R squared 참고사이트
        # http://gouthamanbalaraman.com/blog/calculating-stock-beta.html
        # https://stackoverflow.com/questions/893657/how-do-i-calculate-r-squared-using-python-and-numpy

        # 17.1 Beta
        covariance = np.cov(df['rtn'], df['bm_rtn'])
        # variance는 np.var로 구할수도 있으나, covariance[1,1] 과 같다
        #variance = np.var(m_idx['mkt_rtn'],ddof=1)
        beta = covariance[0,1] / covariance[1,1]
        result['vs_benchmark']['beta'] = beta

        # 17.2 Alpha
        alpha = df['rtn'].mean() - beta*(df['bm_rtn'].mean())
        #연환산
        y_alpha = (1 + alpha) ** NUM_OF_DAYS - 1
        result['vs_benchmark']['alpha'] = y_alpha

        # 17.3 R squared 
        # R2 - numpy_manual

        ypred = alpha + beta * df['bm_rtn']
        SS_res = np.sum(np.power(ypred - df['rtn'],2))
        SS_tot = covariance[0,0] * (len(df) - 1) # SS_TOT is sample_variance*(n-1)
        r_squared = 1. - SS_res/SS_tot
        result['vs_benchmark']['r2'] = r_squared

        # 17.4 correlation
        # 비교를 위해 'rtn', 'mkt_rtn'만 새로운 dataframe 으로 copy
        #new_df = m_idx[['rtn','mkt_rtn']].copy()
        #corr = new_df.corr()
        corr = df['rtn'].corr(df['bm_rtn'])
        result['vs_benchmark']['corr'] = corr
    
    return result

def show_result(rslt) :
    for key in rslt.keys() :
        print("Final Balance : " , int(rslt[key]['final_balance']))
        print("CAGR : ", round(rslt[key]['cagr'] * 100, 5), "%" )
        print("Stdev : ", round(rslt[key]['annlzd_stdev'] * 100, 5), "%" )
        print("Best Year (",rslt[key]['best_y']['year'],") : ", round(rslt[key]['best_y']['rtn'] * 100, 5), "%" )
        print("Worst Year (",rslt[key]['worst_y']['year'],") : ", round(rslt[key]['worst_y']['rtn'] * 100, 5), "%" )
        print("MDD : ", round(rslt[key]['mdd'] * 100, 5), "%" )
        print("Sharpe Ratio : ", round(rslt[key]['sharpe_rto'], 5))
        print("Sortino Ratio : ", round(rslt[key]['sortino_rto'], 5))
        print("Korean MKT Correlation : ", round(rslt[key]['vs_market']['corr'], 5))
        print("Arithmetic Mean (daily) : ", round(rslt[key]['arith_mean'] * 100, 5), "%" )
        print("Arithmetic Mean (annualized) : ", round(rslt[key]['annlzd_arith_mean'] * 100, 5), "%" )
        print("Geometric Mean (daily) : ", round(rslt[key]['geo_mean'] * 100, 5), "%" )
        print("Geometric Mean (annualized) : ", round(rslt[key]['annlzd_geo_mean'] * 100, 5), "%" )
        print("Volatility (daily) : ", round(rslt[key]['stdev'] * 100, 5), "%" )
        print("Volatility (annualized) : ", round(rslt[key]['annlzd_stdev'] * 100, 5), "%" )
        print("Downside Deviation (daily) : ", round(rslt[key]['down_dev'] * 100, 5), "%" )
        print("MDD : ", round(rslt[key]['mdd'] * 100, 5), "%" )
        print("Korean MKT Correlation : ", round(rslt[key]['vs_market']['corr'], 5))
        print("Beta(vs market) : ", round(rslt[key]['vs_market']['beta'], 5))
        print("Alpha(vs market, annualized) : ", round(rslt[key]['vs_market']['alpha']*100, 5),"%")
        print("R2(vs market) : ", round(rslt[key]['vs_market']['r2']*100, 5),"%")
        print("Beta(vs benchmark) : ", round(rslt[key]['vs_benchmark']['beta'], 5))
        print("Alpha(vs benchmark, annualized) : ", round(rslt[key]['vs_benchmark']['alpha']*100, 5),"%")
        print("R2(vs benchmark) : ", round(rslt[key]['vs_benchmark']['r2']*100, 5),"%")
        print("Sharpe Ratio : ", round(rslt[key]['sharpe_rto'], 5))
        print("Sortino Ratio : ", round(rslt[key]['sortino_rto'], 5))
        print("Skewness : ", round(rslt[key]['skewness'], 5))
        print("Excess Kurtosis : ", round(rslt[key]['kurtosis'], 5))
        print("Historical VaR(5%) : ", round(rslt[key]['hist_var']*100, 5),"%")
        print("Analytical VaR(5%) : ", round(rslt[key]['anal_var']*100, 5),"%")
        print("Conditional VaR(5%) : ", round(rslt[key]['c_var']*100, 5),"%")
        print("="*50)

if __name__ == '__main__':
    tm_data = init_analyze(result_df)
    rslt = dict.fromkeys(['backtest','benchmark'])
    rslt['backtest'] = analyze_data(tm_data)

    if 'benchmark' in tm_data.columns:
        tm_data['backtest'] = tm_data['benchmark']
        tm_data['rtn'] = tm_data['bm_rtn']
        tm_data['prev'] = tm_data['bm_prev']
        rslt['benchmark'] = analyze_data(tm_data)
    show_result(rslt)

Final Balance :  1382387
CAGR :  10.11343 %
Stdev :  12.10452 %
Best Year ( 2020 ) :  20.0979 %
Worst Year ( 2021 ) :  7.55422 %
MDD :  -26.74341 %
Sharpe Ratio :  0.70862
Sortino Ratio :  0.97904
Korean MKT Correlation :  0.76477
Arithmetic Mean (daily) :  0.04116 %
Arithmetic Mean (annualized) :  10.92562 %
Geometric Mean (daily) :  0.03824 %
Geometric Mean (annualized) :  10.11343 %
Volatility (daily) :  0.76251 %
Volatility (annualized) :  12.10452 %
Downside Deviation (daily) :  0.55189 %
MDD :  -26.74341 %
Korean MKT Correlation :  0.76477
Beta(vs market) :  0.4745
Alpha(vs market, annualized) :  6.36482 %
R2(vs market) :  58.48767 %
Beta(vs benchmark) :  0.46183
Alpha(vs benchmark, annualized) :  6.03074 %
R2(vs benchmark) :  59.38455 %
Sharpe Ratio :  0.70862
Sortino Ratio :  0.97904
Skewness :  -0.90622
Excess Kurtosis :  15.34846
Historical VaR(5%) :  -0.90545 %
Analytical VaR(5%) :  -1.21307 %
Conditional VaR(5%) :  -1.85039 %
Final Balance :  1382387
CAGR :  8.04527 %
Stdev

In [102]:
# 가장 가까운 다음 날짜(index) 찾기 예제

dt = '2018-01-06'

# method 옵션 - nearest :가장가까운놈, backfill/bfill: Next, pad/ffill : Previous
mm = wght_t.index.get_loc(dt,method='bfill')
find_index = wght_t.index[wght_t.index.get_loc(dt,method='bfill')]

In [103]:
find_index

Timestamp('2018-01-08 00:00:00')

In [68]:
wght_t.to_excel("tttt.xlsx")

In [120]:
# 목표!! 포트폴리오 구성종목, 벤치마크, kospi, riskfree의 일별 가격을 excel로 읽어 json으로 넘기자.
# 각 종목별 엑셀파일을 읽어서 datafram -> json !!
import json

#입력포맷
my_input = {'start_date' : '2018-01-01',
            'init_index' : 1000,
            'init_amt' : 1000000,
            'portfolio' : ['069500','261220','332940','371450','HERO','SPY'],
            'rebalancing' : {
                '2018-01-01' : [15,15,20,25,15,10],
                '2019-01-01' : [15,15,20,25,15,10],
                '2020-01-01' : [15,15,20,25,15,10],
                '2021-01-01' : [15,15,20,25,15,10]}                
            }

# 포트폴리오가격 엑셀에서 가져오기
for item in my_input['portfolio'] :
    # excel 파일 읽기
    infile = 'price_' + item + '.xlsx'
    df = pd.read_excel(infile,dtype='unicode')
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['DATE'] = df['DATE'].dt.strftime('%Y-%m-%d')
    #my_input[item] = df.to_dict('records')
    my_input[item] = df.values.tolist()  # 리스트가 가벼워보여...

# benchmark 엘셀에서 가져오기
infile = "benchmark.xlsx"
tmp_df = pd.read_excel(infile,dtype='unicode')
tmp_df['DATE'] = pd.to_datetime(tmp_df['DATE'])
tmp_df['DATE'] = tmp_df['DATE'].dt.strftime('%Y-%m-%d')
my_input['benchmark'] = tmp_df.values.tolist()

# riskfree 엘셀에서 가져오기
infile = "riskfree.xlsx"
tmp_df = pd.read_excel(infile,dtype='unicode')
tmp_df['DATE'] = pd.to_datetime(tmp_df['DATE'])
tmp_df['DATE'] = tmp_df['DATE'].dt.strftime('%Y-%m-%d')
my_input['riskfree'] = tmp_df.values.tolist()

# 마켓지표(KOSPI) 엘셀에서 가져오기
infile = "kospi.xlsx"
tmp_df = pd.read_excel(infile,dtype='unicode')
tmp_df['DATE'] = pd.to_datetime(tmp_df['DATE'])
tmp_df['DATE'] = tmp_df['DATE'].dt.strftime('%Y-%m-%d')
my_input['kospi'] = tmp_df.values.tolist()

my_json = json.dumps(my_input)
f = open("input.json",'w')
f.write(my_json)
f.close()

In [118]:
my_input.keys()

dict_keys(['start_date', 'init_index', 'init_amt', 'portfolio', 'rebalancing', '069500', '261220', '332940', '371450', 'HERO', 'SPY', 'benchmark', 'riskfree', 'kospi'])

In [116]:
my_input['HERO'][0:10]

[['2019-10-31', '15.03'],
 ['2019-11-01', '15.39'],
 ['2019-11-04', '15.52'],
 ['2019-11-05', '15.5'],
 ['2019-11-06', '15.36'],
 ['2019-11-07', '15.39'],
 ['2019-11-08', '15.48'],
 ['2019-11-11', '15.37'],
 ['2019-11-12', '15.49'],
 ['2019-11-13', '15.41']]

In [119]:
my_json

45.85"], ["2018-04-02", "2444.16"], ["2018-04-03", "2442.43"], ["2018-04-04", "2408.06"], ["2018-04-05", "2437.52"], ["2018-04-06", "2429.58"], ["2018-04-09", "2444.08"], ["2018-04-10", "2450.74"], ["2018-04-11", "2444.22"], ["2018-04-12", "2442.71"], ["2018-04-13", "2455.07"], ["2018-04-16", "2457.49"], ["2018-04-17", "2453.77"], ["2018-04-18", "2479.98"], ["2018-04-19", "2486.1"], ["2018-04-20", "2476.33"], ["2018-04-23", "2474.11"], ["2018-04-24", "2464.14"], ["2018-04-25", "2448.81"], ["2018-04-26", "2475.64"], ["2018-04-27", "2492.4"], ["2018-04-30", "2515.38"], ["2018-05-02", "2505.61"], ["2018-05-03", "2487.25"], ["2018-05-04", "2461.38"], ["2018-05-08", "2449.81"], ["2018-05-09", "2443.98"], ["2018-05-10", "2464.16"], ["2018-05-11", "2477.71"], ["2018-05-14", "2476.11"], ["2018-05-15", "2458.54"], ["2018-05-16", "2459.82"], ["2018-05-17", "2448.45"], ["2018-05-18", "2460.65"], ["2018-05-21", "2465.57"], ["2018-05-23", "2471.91"], ["2018-05-24", "2466.01"], ["2018-05-25", "2460.

In [14]:
tst['columns']

['DATE', '069500']